In [1]:
# =========================================================
# ✅ Cell 0 — Full Environment Reset & Dependency Fix
# =========================================================
!pip uninstall -y transformers accelerate peft bitsandbytes > /dev/null 2>&1

# Install perfectly matched versions
!pip install -U transformers==4.41.2 accelerate==0.31.0 torch datasets seqeval conllu --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 71.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.2 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 2.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 8.8 MB/s eta 0:00:00:00:0100:01
  

In [2]:
# =========================================================
# ✅ Cell 0 — Final Environment Fix (add PEFT)
# =========================================================
!pip install -U peft==0.10.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.7 MB/s eta 0:00:0000:01


In [3]:
import transformers, accelerate, torch
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Torch:", torch.__version__)


Transformers: 4.41.2
Accelerate: 0.31.0
Torch: 2.9.0+cu128


In [4]:
# =========================================================
# ✅ Cell 2 — Copy UD files (EN, HI, UR from /universal-dependencies, BN from /ud-bengali)
# =========================================================
import shutil
from pathlib import Path

# Kaggle dataset locations
KAGGLE_INPUTS = Path("/kaggle/input/universaldependencies")
KAGGLE_BENGALI_INPUTS = Path("/kaggle/input/ud-bengali")

NEEDED = [
    # English
    "en_ewt-ud-train.conllu", "en_ewt-ud-dev.conllu", "en_ewt-ud-test.conllu",
    # Hindi
    "hi_hdtb-ud-train.conllu", "hi_hdtb-ud-dev.conllu", "hi_hdtb-ud-test.conllu",
    # Urdu
    "ur_udtb-ud-train.conllu", "ur_udtb-ud-dev.conllu", "ur_udtb-ud-test.conllu",
    # Bengali (test only)
    "bn_bru-ud-test.conllu"
]

BASE_DIR = Path("./ud_data")
BASE_DIR.mkdir(parents=True, exist_ok=True)

def copy_if_found(fname, src_path):
    dest = BASE_DIR / fname
    if dest.exists():
        return True
    src_file = src_path / fname
    if src_file.exists():
        shutil.copy2(src_file, dest)
        print(f"Copied: {src_file} → {dest}")
        return True
    return False

missing = []
for fname in NEEDED:
    if "bn_bru" in fname:
        ok = copy_if_found(fname, KAGGLE_BENGALI_INPUTS)
    else:
        ok = copy_if_found(fname, KAGGLE_INPUTS)
    if not ok:
        missing.append(fname)

if missing:
    print("❌ Missing files:")
    for m in missing: print(" -", m)
else:
    print("✅ All UD files copied into:", BASE_DIR.resolve())


Copied: /kaggle/input/universaldependencies/en_ewt-ud-train.conllu → ud_data/en_ewt-ud-train.conllu
Copied: /kaggle/input/universaldependencies/en_ewt-ud-dev.conllu → ud_data/en_ewt-ud-dev.conllu
Copied: /kaggle/input/universaldependencies/en_ewt-ud-test.conllu → ud_data/en_ewt-ud-test.conllu
Copied: /kaggle/input/universaldependencies/hi_hdtb-ud-train.conllu → ud_data/hi_hdtb-ud-train.conllu
Copied: /kaggle/input/universaldependencies/hi_hdtb-ud-dev.conllu → ud_data/hi_hdtb-ud-dev.conllu
Copied: /kaggle/input/universaldependencies/hi_hdtb-ud-test.conllu → ud_data/hi_hdtb-ud-test.conllu
Copied: /kaggle/input/universaldependencies/ur_udtb-ud-train.conllu → ud_data/ur_udtb-ud-train.conllu
Copied: /kaggle/input/universaldependencies/ur_udtb-ud-dev.conllu → ud_data/ur_udtb-ud-dev.conllu
Copied: /kaggle/input/universaldependencies/ur_udtb-ud-test.conllu → ud_data/ur_udtb-ud-test.conllu
Copied: /kaggle/input/ud-bengali/bn_bru-ud-test.conllu → ud_data/bn_bru-ud-test.conllu
✅ All UD files copi

In [4]:
# =========================================================
# ✅ Cell 3 — Load and parse CoNLL-U files (UPOS tags only)
# =========================================================
from typing import List, Tuple, Dict
from pathlib import Path

TRAIN_LANGS = ["en_ewt", "hi_hdtb", "ur_udtb"]
ZERO_SHOT_TEST_LANG = "bn_bcc"  # internal key for BN
BASE_DIR = Path("./ud_data")

UD_LOCAL = {
    "en_ewt": {"train": "en_ewt-ud-train.conllu", "dev": "en_ewt-ud-dev.conllu", "test": "en_ewt-ud-test.conllu"},
    "hi_hdtb": {"train": "hi_hdtb-ud-train.conllu", "dev": "hi_hdtb-ud-dev.conllu", "test": "hi_hdtb-ud-test.conllu"},
    "ur_udtb": {"train": "ur_udtb-ud-train.conllu", "dev": "ur_udtb-ud-dev.conllu", "test": "ur_udtb-ud-test.conllu"},
    "bn_bcc": {"test": "bn_bru-ud-test.conllu"},
}

def read_conllu_upos(path: Path) -> List[Tuple[List[str], List[str]]]:
    sents, words, upos = [], [], []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    sents.append((words, upos))
                    words, upos = [], []
                continue
            if line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) != 10: continue
            idx = parts[0]
            if "-" in idx or "." in idx:  # skip multiword/empty
                continue
            words.append(parts[1])
            upos.append(parts[3])
    if words:
        sents.append((words, upos))
    return sents

def load_split(lang: str, split: str):
    assert lang in UD_LOCAL
    path = BASE_DIR / UD_LOCAL[lang][split]
    return read_conllu_upos(path)

def build_upos_vocab(lang_keys: List[str]):
    tagset = set()
    for lk in lang_keys:
        for sp in UD_LOCAL[lk]:
            for _, tags in load_split(lk, sp):
                tagset.update(tags)
    id2tag = sorted(tagset)
    tag2id = {t: i for i, t in enumerate(id2tag)}
    return tag2id, id2tag

tag2id, id2tag = build_upos_vocab(TRAIN_LANGS + [ZERO_SHOT_TEST_LANG])
NUM_TAGS = len(id2tag)
print("NUM_TAGS:", NUM_TAGS)
print("UPOS tags:", id2tag)


NUM_TAGS: 17
UPOS tags: ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']


In [5]:
# =========================================================
# ✅ Cell 4 — Model Setup (XLM-R)
# =========================================================
import torch, random, numpy as np
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    get_linear_schedule_with_warmup, DataCollatorForTokenClassification
)

SEED = 42
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

MODEL_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=NUM_TAGS).to(device)
print("✅ Loaded", MODEL_NAME, "with", NUM_TAGS, "labels")


2025-10-29 17:08:31.020957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761757711.243732      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761757711.300367      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cuda


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded xlm-roberta-base with 17 labels


In [7]:
# =========================================================
# ✅ Cell 5 — Tokenization + Alignment
# =========================================================
from torch.utils.data import Dataset, DataLoader

MAX_LEN = 256

def tokenize_and_align_labels(sentences, upos_tags, tag2id):
    enc = tokenizer(
        sentences, is_split_into_words=True,
        truncation=True, padding=False, max_length=MAX_LEN
    )
    labels = []
    for i, tags in enumerate(upos_tags):
        word_ids = enc.word_ids(i)
        label_ids, prev = [], None
        for w_id in word_ids:
            if w_id is None:
                label_ids.append(-100)
            elif w_id != prev:
                label_ids.append(tag2id[tags[w_id]])
            else:
                label_ids.append(-100)
            prev = w_id
        labels.append(label_ids)
    enc["labels"] = labels
    return enc

class UPOSDataset(Dataset):
    def __init__(self, sents_tags):
        self.sents = [w for w, _ in sents_tags]
        self.tags = [t for _, t in sents_tags]
        self.enc = tokenize_and_align_labels(self.sents, self.tags, tag2id)
    def __len__(self): return len(self.sents)
    def __getitem__(self, i): return {k: torch.tensor(v[i]) for k,v in self.enc.items()}

data_collator = DataCollatorForTokenClassification(tokenizer)


In [8]:
# =========================================================
# ✅ Cell 6 — Build datasets/loaders
# =========================================================
BATCH_SIZE = 16

def make_dataset(lang, split): return UPOSDataset(load_split(lang, split))

# Train/dev
en_train, hi_train, ur_train = make_dataset("en_ewt","train"), make_dataset("hi_hdtb","train"), make_dataset("ur_udtb","train")
en_dev, hi_dev, ur_dev = make_dataset("en_ewt","dev"), make_dataset("hi_hdtb","dev"), make_dataset("ur_udtb","dev")
# Test
bn_test = make_dataset("bn_bcc","test")

def make_loader(ds, shuffle=False):
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=data_collator)

ldr = {
    "en_train": make_loader(en_train, True),
    "hi_train": make_loader(hi_train, True),
    "ur_train": make_loader(ur_train, True),
    "en_dev": make_loader(en_dev), "hi_dev": make_loader(hi_dev), "ur_dev": make_loader(ur_dev),
    "bn_test": make_loader(bn_test)
}
for k,v in ldr.items():
    print(f"{k}: {len(v)} batches")


en_train: 784 batches
hi_train: 832 batches
ur_train: 253 batches
en_dev: 126 batches
hi_dev: 104 batches
ur_dev: 35 batches
bn_test: 4 batches


In [9]:
# =========================================================
# ✅ Cell 7 — Metrics (seqeval)
# =========================================================
from seqeval.metrics import classification_report, f1_score, accuracy_score

def decode_predictions(logits, labels):
    preds = logits.argmax(-1)
    y_true, y_pred = [], []
    for p, l in zip(preds, labels):
        true_seq, pred_seq = [], []
        for pi, li in zip(p.tolist(), l.tolist()):
            if li == -100: continue
            true_seq.append(id2tag[li])
            pred_seq.append(id2tag[pi])
        if true_seq:
            y_true.append(true_seq); y_pred.append(pred_seq)
    return y_true, y_pred

@torch.no_grad()
def evaluate_loader(model, loader, desc="eval"):
    model.eval()
    all_true, all_pred = [], []
    for batch in loader:
        batch = {k:v.to(device) for k,v in batch.items()}
        out = model(**batch)
        yt, yp = decode_predictions(out.logits.cpu(), batch["labels"].cpu())
        all_true += yt; all_pred += yp
    acc = accuracy_score(all_true, all_pred)
    f1 = f1_score(all_true, all_pred)
    return {"acc": acc, "f1": f1, "report": classification_report(all_true, all_pred, digits=4)}


In [12]:
# =========================================================
# ✅ Cell 8 — Round-robin fine-tuning (EN→HI→UR)
# =========================================================
from torch.optim import AdamW
import itertools

EPOCHS = 6
LR = 1e-5
MAX_GRAD_NORM = 1.0
WARMUP_RATIO = 0.06

def cycle_loader(dl):
    while True:
        for b in dl: yield b

def make_epoch_iters():
    en_it, hi_it, ur_it = map(cycle_loader, [ldr["en_train"], ldr["hi_train"], ldr["ur_train"]])
    per_lang = min(len(ldr["en_train"]), len(ldr["hi_train"]), len(ldr["ur_train"]))
    return en_it, hi_it, ur_it, per_lang

steps_per_epoch = min(len(ldr["en_train"]), len(ldr["hi_train"]), len(ldr["ur_train"])) * 3
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

optimizer = AdamW(model.parameters(), lr=LR)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

best_f1, best_ckpt = -1, "xlmr_pos_roundrobin_best.ckpt"

for epoch in range(1, EPOCHS+1):
    model.train()
    en_it, hi_it, ur_it, per_lang = make_epoch_iters()
    lang_cycle = itertools.cycle(["en","hi","ur"])
    total_batches = per_lang * 3
    losses = []

    for step in range(total_batches):
        lang = next(lang_cycle)
        batch = next(en_it) if lang=="en" else next(hi_it) if lang=="hi" else next(ur_it)
        batch = {k:v.to(device) for k,v in batch.items()}
        out = model(**batch)
        loss = out.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        optimizer.step(); scheduler.step(); optimizer.zero_grad(set_to_none=True)
        losses.append(loss.item())
        if (step+1)%100==0 or (step+1)==total_batches:
            print(f"Epoch {epoch} Step {step+1}/{total_batches} | Loss {np.mean(losses):.4f}")

    # Evaluate
    en_m, hi_m, ur_m = evaluate_loader(model, ldr["en_dev"]), evaluate_loader(model, ldr["hi_dev"]), evaluate_loader(model, ldr["ur_dev"])
    avg_f1 = (en_m["f1"] + hi_m["f1"] + ur_m["f1"]) / 3
    print(f"\nEpoch {epoch} Dev F1s → EN={en_m['f1']:.4f}, HI={hi_m['f1']:.4f}, UR={ur_m['f1']:.4f}, AVG={avg_f1:.4f}")

    if avg_f1 > best_f1:
        best_f1 = avg_f1
        torch.save(model.state_dict(), best_ckpt)
        print(f"✅ Saved new best (AVG F1={best_f1:.4f}) → {best_ckpt}")

print("🎯 Training done. Best AVG F1 =", best_f1)


Epoch 1 Step 100/759 | Loss 0.0636
Epoch 1 Step 200/759 | Loss 0.0627
Epoch 1 Step 300/759 | Loss 0.0636
Epoch 1 Step 400/759 | Loss 0.0647
Epoch 1 Step 500/759 | Loss 0.0651
Epoch 1 Step 600/759 | Loss 0.0662
Epoch 1 Step 700/759 | Loss 0.0660
Epoch 1 Step 759/759 | Loss 0.0659

Epoch 1 Dev F1s → EN=0.9723, HI=0.9750, UR=0.9360, AVG=0.9611
✅ Saved new best (AVG F1=0.9611) → xlmr_pos_roundrobin_best.ckpt
Epoch 2 Step 100/759 | Loss 0.0622
Epoch 2 Step 200/759 | Loss 0.0632
Epoch 2 Step 300/759 | Loss 0.0625
Epoch 2 Step 400/759 | Loss 0.0631
Epoch 2 Step 500/759 | Loss 0.0624
Epoch 2 Step 600/759 | Loss 0.0627
Epoch 2 Step 700/759 | Loss 0.0629
Epoch 2 Step 759/759 | Loss 0.0628

Epoch 2 Dev F1s → EN=0.9740, HI=0.9735, UR=0.9392, AVG=0.9622
✅ Saved new best (AVG F1=0.9622) → xlmr_pos_roundrobin_best.ckpt
Epoch 3 Step 100/759 | Loss 0.0589
Epoch 3 Step 200/759 | Loss 0.0559
Epoch 3 Step 300/759 | Loss 0.0562
Epoch 3 Step 400/759 | Loss 0.0553
Epoch 3 Step 500/759 | Loss 0.0559
Epoch 3 S

In [13]:
# =========================================================
# ✅ Cell 9 — evaluation on Bangla
# =========================================================
BEST_CKPT = "xlmr_pos_roundrobin_best.ckpt"
if Path(BEST_CKPT).exists():
    model.load_state_dict(torch.load(BEST_CKPT, map_location=device))
    print("Loaded best model:", BEST_CKPT)
else:
    print("⚠️ No checkpoint found, using current model.")

bn_metrics = evaluate_loader(model, ldr["bn_test"])
print("\n===== Bangla Evaluation =====")
print(f"Accuracy: {bn_metrics['acc']:.4f}")
print(f"F1: {bn_metrics['f1']:.4f}\n")
print("Detailed Report:\n", bn_metrics["report"])


Loaded best model: xlmr_pos_roundrobin_best.ckpt

===== Bangla Evaluation =====
Accuracy: 0.8594
F1: 0.8182

Detailed Report:
               precision    recall  f1-score   support

         ART     0.6364    0.7778    0.7000         9
        CONJ     0.5000    0.5000    0.5000         2
          DJ     0.6429    0.7500    0.6923        12
          DP     1.0000    0.5000    0.6667         2
          DV     0.6000    0.2500    0.3529        12
         ERB     0.8571    0.8077    0.8317        52
          ET     0.5000    0.2308    0.3158        13
         NTJ     0.7500    1.0000    0.8571         6
         OUN     0.8200    0.8200    0.8200        50
         RON     0.7358    0.9070    0.8125        43
        ROPN     0.5714    1.0000    0.7273         4
          UM     0.6667    1.0000    0.8000         2
        UNCT     1.0000    1.0000    1.0000        73
          UX     0.7500    0.5000    0.6000         6

   micro avg     0.8182    0.8182    0.8182       286
   macr

In [5]:
# ============================
# 🇬🇧🇮🇳🇵🇰 → 🇧🇩  Multi-Source POS (XLM-R)
# EN+HI+UR train/dev  →  BN test
# Sweep: LR × Epochs ; Multi-seed ; Reports + Confusion + Curves
# ============================
import os
# Quiet noisy libs BEFORE importing HF/Torch
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
import torch
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from conllu import parse_incr
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    Trainer, TrainingArguments, DataCollatorForTokenClassification, set_seed
)
from transformers.trainer_utils import EvalPrediction
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from itertools import product
from pathlib import Path

# Harmless warnings
warnings.filterwarnings("ignore", message="Some weights of")
warnings.filterwarnings("ignore", message="checkpoint")
warnings.filterwarnings("ignore", message="AdamW is deprecated")

# ===== Universal POS labels (UD) =====
LABEL_LIST = ['ADJ','ADP','ADV','AUX','CCONJ','DET','INTJ','NOUN','NUM',
              'PART','PRON','PROPN','PUNCT','SCONJ','SYM','VERB','X']
LABEL2ID = {t:i for i,t in enumerate(LABEL_LIST)}
ID2LABEL = {i:t for t,i in LABEL2ID.items()}

# ===== Config =====
BASE_CFG = dict(
    model_name="xlm-roberta-base",
    max_len=128,
    train_bs=8,
    eval_bs=16,
    # small sweep
    sweep_lrs=[3e-5],#1e-5
    sweep_epochs=[5],#3
    # multi-seed
    seeds=[42],#7,2024
)

# ------------------------- Utility Functions -------------------------

def _set_all_seeds(seed:int):
    set_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def _load_ud(file_path:str) -> Dataset:
    """Read a UD .conllu file → HF Dataset with tokens and upos."""
    cache = file_path + ".cache"
    if os.path.exists(cache):
        return Dataset.load_from_disk(cache)
    rows = []
    with open(file_path, "r", encoding="utf-8") as f:
        for tokenlist in parse_incr(f):
            rows.append({"tokens":[t["form"] for t in tokenlist],
                         "upos":[t["upos"] for t in tokenlist]})
    ds = Dataset.from_pandas(pd.DataFrame(rows))
    ds.save_to_disk(cache)
    return ds

def _to_ids(examples):
    """Map UPOS tags → integer IDs."""
    return {"upos":[[LABEL2ID.get(tag, LABEL2ID['X']) for tag in tags] for tags in examples["upos"]]}

def _tokenize_fn(tokenizer, max_len):
    """Tokenize & align UPOS tags (only first subword is labeled; others = -100)."""
    def f(examples):
        tok = tokenizer(
            examples["tokens"], truncation=True, padding="max_length",
            max_length=max_len, is_split_into_words=True
        )
        labels = []
        for i, lab in enumerate(examples["upos"]):
            word_ids = tok.word_ids(batch_index=i)
            prev = None; ids = []
            for w in word_ids:
                if w is None:
                    ids.append(-100)
                elif w != prev:
                    ids.append(lab[w])
                else:
                    ids.append(-100)
                prev = w
            # hard pad to max_len (defensive; tokenizer already pads)
            ids += [-100]*(max_len - len(ids))
            labels.append(ids[:max_len])
        tok["labels"] = labels
        return tok
    return f

def _flatten_preds_labels(pred_logits, dataset):
    """Flatten predictions/labels across tokens, ignoring -100."""
    preds = np.argmax(pred_logits, axis=2)
    y_true_ids, y_pred_ids = [], []
    for i in range(len(dataset)):
        labels = dataset[i]["labels"]
        mask = np.array(labels) != -100
        y_true_ids.extend(np.array(labels)[mask].tolist())
        y_pred_ids.extend(np.array(preds[i])[mask].tolist())
    return y_true_ids, y_pred_ids

def _metrics_from_ids(y_true_ids, y_pred_ids):
    """Compute accuracy, macro-F1, weighted-F1, full report, and confusion matrix."""
    y_true = [ID2LABEL[i] for i in y_true_ids]
    y_pred = [ID2LABEL[i] for i in y_pred_ids]
    acc = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, labels=LABEL_LIST, average="macro", zero_division=0)
    weighted_f1 = f1_score(y_true, y_pred, labels=LABEL_LIST, average="weighted", zero_division=0)
    report = classification_report(y_true, y_pred, labels=LABEL_LIST, digits=4, zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=LABEL_LIST)  # counts
    return acc, macro_f1, weighted_f1, report, cm

def _plot_confusion(cm_norm, title, out_png):
    plt.figure(figsize=(12,10))
    sns.heatmap(cm_norm, annot=False, cmap="Blues",
                xticklabels=LABEL_LIST, yticklabels=LABEL_LIST)
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

def _compute_metrics(eval_pred: EvalPrediction):
    """For Trainer: token accuracy (ignoring -100)."""
    preds = np.argmax(eval_pred.predictions, axis=2)
    labels = eval_pred.label_ids
    mask = labels != -100
    total = mask.sum()
    correct = ((preds == labels) & mask).sum()
    acc = float(correct) / float(total) if total > 0 else 0.0
    return {"accuracy": acc}

def _plot_learning_curve_from_logs(log_history, out_png, title="Learning Curve"):
    epochs, eval_loss, eval_acc = [], [], []
    train_epochs, train_loss = [], []
    for rec in log_history:
        if "eval_loss" in rec:
            epochs.append(rec.get("epoch"))
            eval_loss.append(rec["eval_loss"])
            eval_acc.append(rec.get("eval_accuracy"))
        elif "loss" in rec and "epoch" in rec:
            train_epochs.append(rec["epoch"])
            train_loss.append(rec["loss"])
    plt.figure(figsize=(10,6))
    if train_epochs:
        plt.plot(train_epochs, train_loss, label="train_loss", marker="o")
    if epochs:
        plt.plot(epochs, eval_loss, label="eval_loss", marker="o")
    if epochs and any(a is not None for a in eval_acc):
        ax = plt.gca()
        ax2 = ax.twinx()
        ax2.plot(epochs, eval_acc, label="eval_accuracy", marker="s", linestyle="--", color="tab:green")
        ax2.set_ylabel("Eval Accuracy")
        ax2.legend(loc="lower right")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(title)
    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

# ------------------------- Main Experiment -------------------------

def run_pos_tagging_multisource():
    """
    Train on EN+HI+UR (train) ; validate on EN+HI+UR (dev concatenated).
    Zero-shot evaluate on BN test.
    Multi-seed + hyperparameter sweep.
    """
    print("\n" + "="*50 + "\nMULTI-SOURCE POS TAGGING (EN+HI+UR → BN)\n" + "="*50)
    out_root = Path("results_msweep_multisrc"); out_root.mkdir(exist_ok=True, parents=True)

    # ===== 1️⃣ Load & preprocess datasets from ud_data =====
    DATA_DIR = Path("./ud_data")
    print("📂 Loading datasets from:", DATA_DIR.resolve())

    en_train = _load_ud(str(DATA_DIR / "en_ewt-ud-train.conllu")).map(_to_ids, batched=True)
    hi_train = _load_ud(str(DATA_DIR / "hi_hdtb-ud-train.conllu")).map(_to_ids, batched=True)
    ur_train = _load_ud(str(DATA_DIR / "ur_udtb-ud-train.conllu")).map(_to_ids, batched=True)

    en_dev = _load_ud(str(DATA_DIR / "en_ewt-ud-dev.conllu")).map(_to_ids, batched=True)
    hi_dev = _load_ud(str(DATA_DIR / "hi_hdtb-ud-dev.conllu")).map(_to_ids, batched=True)
    ur_dev = _load_ud(str(DATA_DIR / "ur_udtb-ud-dev.conllu")).map(_to_ids, batched=True)

    # Bengali (zero-shot test)
    bn_test = _load_ud(str(DATA_DIR / "bn_bru-ud-test.conllu")).map(_to_ids, batched=True)

    # Concatenate multi-source train/dev
    from datasets import concatenate_datasets
    train_ds = concatenate_datasets([en_train, hi_train, ur_train])
    dev_ds   = concatenate_datasets([en_dev, hi_dev, ur_dev])

    print(f"✅ Loaded: Train={len(train_ds)}  Dev={len(dev_ds)}  BN-Test={len(bn_test)}")

    # ===== 2️⃣ Tokenize =====
    tokenizer = AutoTokenizer.from_pretrained(BASE_CFG["model_name"])
    tok_fn = _tokenize_fn(tokenizer, BASE_CFG["max_len"])

    print("🧩 Tokenizing datasets…")
    train_tok = train_ds.map(tok_fn, batched=True, remove_columns=train_ds.column_names)
    dev_tok   = dev_ds.map(tok_fn,   batched=True, remove_columns=dev_ds.column_names)
    bn_test_tok = bn_test.map(tok_fn, batched=True, remove_columns=bn_test.column_names)

    # (rest of script remains unchanged — zero-shot, sweep, etc.)


    # ===== 2) Zero-shot baseline on BN-test =====
    print("\n--- ZERO-SHOT (bn_test) ---")
    zs_model = AutoModelForTokenClassification.from_pretrained(
        BASE_CFG["model_name"], num_labels=len(LABEL_LIST),
        id2label=ID2LABEL, label2id=LABEL2ID, ignore_mismatched_sizes=True
    )
    zs_trainer = Trainer(
        model=zs_model,
        args=TrainingArguments(
            output_dir=str(out_root/"zero_shot"),
            per_device_eval_batch_size=BASE_CFG["eval_bs"],
            report_to="none",
            dataloader_num_workers=2,
            dataloader_pin_memory=True,
            fp16=torch.cuda.is_available(),
        ),
        data_collator=DataCollatorForTokenClassification(tokenizer)
    )
    zs_pred = zs_trainer.predict(bn_test_tok)
    zs_true, zs_pred_ids = _flatten_preds_labels(zs_pred.predictions, bn_test_tok)
    zs_acc, zs_mf1, zs_wf1, zs_report, zs_cm = _metrics_from_ids(zs_true, zs_pred_ids)
    (out_root/"zero_shot").mkdir(exist_ok=True, parents=True)
    with open(out_root/"zero_shot"/"bn_test_report.txt","w",encoding="utf-8") as f:
        f.write(f"acc={zs_acc:.6f}  macroF1={zs_mf1:.6f}  weightedF1={zs_wf1:.6f}\n\n{zs_report}")
    zs_cm_norm = zs_cm / (zs_cm.sum(axis=1, keepdims=True) + 1e-9)
    _plot_confusion(zs_cm_norm, "Zero-shot Confusion (BN test)", out_root/"zero_shot"/"bn_confusion.png")
    del zs_model; torch.cuda.empty_cache()

    # ===== 3) Multi-seed tiny sweep + learning curves =====
    runs = []
    aggregate_cm_counts = np.zeros((len(LABEL_LIST), len(LABEL_LIST)), dtype=np.int64)

    for seed in BASE_CFG["seeds"]:
        print(f"\n===== SEED {seed} =====")
        _set_all_seeds(seed)
        best = dict(macro_f1=-1, lr=None, epochs=None, trainer=None, model_dir=None)

        for lr, epochs in product(BASE_CFG["sweep_lrs"], BASE_CFG["sweep_epochs"]):
            print(f"  >> Try LR={lr}, Epochs={epochs}")
            _set_all_seeds(seed)
            model = AutoModelForTokenClassification.from_pretrained(
                BASE_CFG["model_name"], num_labels=len(LABEL_LIST),
                id2label=ID2LABEL, label2id=LABEL2ID, ignore_mismatched_sizes=True
            )
            args = TrainingArguments(
                output_dir=str(out_root/f"tmp_s{seed}_lr{lr}_ep{epochs}"),
                learning_rate=lr,
                per_device_train_batch_size=BASE_CFG["train_bs"],
                per_device_eval_batch_size=BASE_CFG["eval_bs"],
                num_train_epochs=epochs,
                eval_strategy="epoch",
                save_strategy="no",
                logging_strategy="epoch",
                report_to="none",
                fp16=torch.cuda.is_available(),
                remove_unused_columns=True,
                dataloader_num_workers=2,
                dataloader_pin_memory=True,
                seed=seed,
            )
            trainer = Trainer(
                model=model, args=args,
                train_dataset=train_tok, eval_dataset=dev_tok,
                data_collator=DataCollatorForTokenClassification(tokenizer),
                compute_metrics=_compute_metrics
            )
            trainer.train()

            # Dev evaluation for selection (macro-F1 on concatenated EN+HI+UR dev)
            dev_pred = trainer.predict(dev_tok)
            y_true_ids, y_pred_ids = _flatten_preds_labels(dev_pred.predictions, dev_tok)
            dev_macro_f1 = f1_score([ID2LABEL[i] for i in y_true_ids],
                                    [ID2LABEL[i] for i in y_pred_ids],
                                    labels=LABEL_LIST, average="macro", zero_division=0)
            print(f"     dev macro-F1 = {dev_macro_f1:.4f}")

            if dev_macro_f1 > best["macro_f1"]:
                best.update(macro_f1=dev_macro_f1, lr=lr, epochs=epochs, trainer=trainer, model_dir=args.output_dir)
            else:
                del trainer.model; del trainer; torch.cuda.empty_cache()

        # Use best trainer/model for this seed → Final EN+HI+UR-dev & BN-test
        seed_dir = out_root/f"seed_{seed}"; Path(seed_dir).mkdir(exist_ok=True, parents=True)
        print(f"Best for seed {seed}: LR={best['lr']}  Epochs={best['epochs']}  (dev macro-F1={best['macro_f1']:.4f})")

        # Save best model + tokenizer
        best_dir = Path(seed_dir) / "best_model"
        best["trainer"].save_model(str(best_dir))
        tokenizer.save_pretrained(str(best_dir))
        print(f"Saved best model for seed {seed} to: {best_dir}")

        # Learning curve from best trainer logs (multi-source dev)
        log_hist = best["trainer"].state.log_history
        pd.DataFrame(log_hist).to_csv(Path(seed_dir)/"training_log_history.csv", index=False)
        _plot_learning_curve_from_logs(
            log_hist,
            Path(seed_dir)/"learning_curve.png",
            title=f"Learning Curve (EN+HI+UR dev) — seed {seed}, lr={best['lr']}, ep={best['epochs']}"
        )

        # Multi-source dev final report
        dev_pred = best["trainer"].predict(dev_tok)
        dev_true, dev_pred_ids = _flatten_preds_labels(dev_pred.predictions, dev_tok)
        dev_acc, dev_mf1, dev_wf1, dev_report, dev_cm = _metrics_from_ids(dev_true, dev_pred_ids)
        with open(Path(seed_dir)/"multi_dev_report.txt","w",encoding="utf-8") as f:
            f.write(f"acc={dev_acc:.6f}  macroF1={dev_mf1:.6f}  weightedF1={dev_wf1:.6f}\n\n{dev_report}")
        dev_cm_norm = dev_cm / (dev_cm.sum(axis=1, keepdims=True) + 1e-9)
        _plot_confusion(dev_cm_norm, f"EN+HI+UR-dev Confusion (seed {seed})", Path(seed_dir)/"multi_dev_confusion.png")

        # BN-test final report (held-out, zero-shot target)
        test_pred = best["trainer"].predict(bn_test_tok)
        test_true, test_pred_ids = _flatten_preds_labels(test_pred.predictions, bn_test_tok)
        test_acc, test_mf1, test_wf1, test_report, test_cm = _metrics_from_ids(test_true, test_pred_ids)
        with open(Path(seed_dir)/"bn_test_report.txt","w",encoding="utf-8") as f:
            f.write(f"acc={test_acc:.6f}  macroF1={test_mf1:.6f}  weightedF1={test_wf1:.6f}\n\n{test_report}")
        test_cm_norm = test_cm / (test_cm.sum(axis=1, keepdims=True) + 1e-9)
        _plot_confusion(test_cm_norm, f"BN-test Confusion (seed {seed})", Path(seed_dir)/"bn_confusion.png")

        # Accumulate results
        runs.append(dict(seed=seed,
                         multisrc_dev=dict(acc=dev_acc, macro_f1=dev_mf1, weighted_f1=dev_wf1),
                         bn=dict(acc=test_acc, macro_f1=test_mf1, weighted_f1=test_wf1)))
        aggregate_cm_counts += test_cm

        del best["trainer"].model; del best["trainer"]; torch.cuda.empty_cache()

    # ===== 4) Aggregate across seeds =====
    def _agg(metric):
        dev_vals = [r["multisrc_dev"][metric] for r in runs]
        bn_vals  = [r["bn"][metric] for r in runs]
        return (np.mean(dev_vals), np.std(dev_vals), np.mean(bn_vals), np.std(bn_vals))

    mean_dev_acc, std_dev_acc, mean_bn_acc, std_bn_acc = _agg("acc")
    mean_dev_mf1, std_dev_mf1, mean_bn_mf1, std_bn_mf1 = _agg("macro_f1")
    mean_dev_wf1, std_dev_wf1, mean_bn_wf1, std_bn_wf1 = _agg("weighted_f1")

    agg_cm_norm = aggregate_cm_counts / (aggregate_cm_counts.sum(axis=1, keepdims=True) + 1e-9)
    _plot_confusion(agg_cm_norm, "BN-test Confusion (Aggregate over seeds)", out_root/"bn_confusion_aggregate.png")

    summary = f"""FINAL SUMMARY over {len(runs)} seed(s)
EN+HI+UR dev : acc={mean_dev_acc:.4f}±{std_dev_acc:.4f} | macroF1={mean_dev_mf1:.4f}±{std_dev_mf1:.4f} | weightedF1={mean_dev_wf1:.4f}±{std_dev_wf1:.4f}
BN-test      : acc={mean_bn_acc:.4f}±{std_bn_acc:.4f} | macroF1={mean_bn_mf1:.4f}±{std_bn_mf1:.4f} | weightedF1={mean_bn_wf1:.4f}±{std_bn_wf1:.4f}

Zero-shot BN-test (single run): acc={zs_acc:.4f} | macroF1={zs_mf1:.4f} | weightedF1={zs_wf1:.4f}
"""
    print("\n" + summary)
    out_root.joinpath("summary.txt").write_text(summary, encoding="utf-8")

    return {
        "zero_shot_bn": dict(acc=zs_acc, macro_f1=zs_mf1, weighted_f1=zs_wf1),
        "runs": runs,
        "mean_std": {
            "multisrc_dev": {"acc": (mean_dev_acc, std_dev_acc), "macro_f1": (mean_dev_mf1, std_dev_mf1), "weighted_f1": (mean_dev_wf1, std_dev_wf1)},
            "bn_test": {"acc": (mean_bn_acc, std_bn_acc), "macro_f1": (mean_bn_mf1, std_bn_mf1), "weighted_f1": (mean_bn_wf1, std_bn_wf1)},
        }
    }

if __name__ == "__main__":
    run_pos_tagging_multisource()


E0000 00:00:1761766434.562925      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761766434.690271      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



MULTI-SOURCE POS TAGGING (EN+HI+UR → BN)
📂 Loading datasets from: /kaggle/working/ud_data


Saving the dataset (0/1 shards):   0%|          | 0/12544 [00:00<?, ? examples/s]

Map:   0%|          | 0/12544 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13306 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.182500,0.093423,0.969870
2,0.075800,0.087061,0.972792
3,0.054100,0.088025,0.974545
4,0.040200,0.092146,0.974624
5,0.030300,0.097849,0.975408


     dev macro-F1 = 0.9687
Best for seed 42: LR=3e-05  Epochs=5  (dev macro-F1=0.9687)
Saved best model for seed 42 to: results_msweep_multisrc/seed_42/best_model


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



FINAL SUMMARY over 1 seed(s)
EN+HI+UR dev : acc=0.9754±0.0000 | macroF1=0.9687±0.0000 | weightedF1=0.9754±0.0000
BN-test      : acc=0.8562±0.0000 | macroF1=0.5617±0.0000 | weightedF1=0.8428±0.0000

Zero-shot BN-test (single run): acc=0.0187 | macroF1=0.0022 | weightedF1=0.0007

